<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| no. Paquete | T. llegada | T. salida | Tiempo de arribo | Inicio de servicio | Hora de salida | Tiempo en fila | Tiempo en el sistema |
|-------------|------------|-----------|-------------------|---------------------|-----------------|----------------|------------------------|
| 1           | 2          | 12        | 2                 | 2                   | 14              | 0              | 12                     |
| 2           | 5          | 10        | 7                 | 14                  | 24              | 7              | 17                     |
| 3           | 1          | 16        | 8                 | 24                  | 40              | 16             | 32                     |
| 4           | 4          | 9         | 12                | 40                  | 49              | 28             | 37                     |
| 5           | 1          | 10        | 13                | 49                  | 59              | 36             | 46                     |
| 6           | 3          | 13        | 16                | 59                  | 72              | 43             | 56                     |
| 7           | 3          | 17        | 19                | 72                  | 89              | 53             | 70                     |
| 8           | 2          | 10        | 21                | 89                  | 99              | 68             | 78                     |
| 9           | 4          | 8         | 25                | 99                  | 107             | 74             | 82                     |
| 10          | 5          | 12        | 30                | 107                 | 119             | 77             | 89                     |


In [3]:
import random
import math

In [4]:
# Datos de la tabla
tiempos_llegada = [2, 5, 1, 4, 1, 3, 3, 2, 4, 5]      # T. llegada (interarrivals)
tiempos_servicio = [12, 10, 16, 9, 10, 13, 17, 10, 8, 12]  # T. salida (tiempos de servicio)

# Medias (esto es lo que tu profe llama lambda y mu)
lambda_media = sum(tiempos_llegada) / len(tiempos_llegada)   # debería dar 3
mu_media = sum(tiempos_servicio) / len(tiempos_servicio)     # debería dar 11.7

print("Lambda (media de tiempos entre llegada):", lambda_media)
print("Mu (media de tiempos de servicio):", mu_media)

Lambda (media de tiempos entre llegada): 3.0
Mu (media de tiempos de servicio): 11.7
